# Smoke Mortality Analysis

This notebook analyzes the impact of wildfire smoke on mortality rates globally. It compares baseline PM2.5 concentrations with no-fire scenarios to estimate the number of deaths attributable to wildfire smoke.

## Data Requirements

This notebook requires the following data files in the `./data/` directory:
- PM2.5 concentration files:
  - `CESM_09x125_PM25_2050_RCP85.nc`: Baseline PM2.5 concentrations for 2050
  - `CESM_09x125_PM25_2050_RCP85_NoFire.nc`: No-fire PM2.5 concentrations for 2050
  - `CESM_09x125_PM25_2000_Baseline.nc`: Baseline PM2.5 concentrations for 2000
  - `CESM_09x125_PM25_2000_BaseLine_NoFire.nc`: No-fire PM2.5 concentrations for 2000
  - `CESM_09x125_PM25_2100_RCP85.nc`: Baseline PM2.5 concentrations for 2100
  - `CESM_09x125_PM25_2100_RCP85_NoFire.nc`: No-fire PM2.5 concentrations for 2100
- `death_rate.nc`: Death rates by location (output from country_process_github.ipynb)
- Population projection files:
  - `ssp2_2000_matching.nc`: Population projections for 2000
  - `ssp2_2050_matching.nc`: Population projections for 2050
  - `ssp2_2100_matching.nc`: Population projections for 2100
- Natural Earth shapefiles for country boundaries

The notebook will output CSV files with smoke-related mortality estimates by country.

In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import cartopy.io.shapereader as shpreader
from shapely.geometry.polygon import Polygon
from shapely.geometry import shape
import pandas as pd
from shapely.geometry import Point, Polygon
from cartopy.io.shapereader import Reader
from rapidfuzz import process
import matplotlib.colors as mcolors

## Setup Data Directory

Check if the data directory exists and contains required files.

In [2]:
# Define the data directory relative to the notebook location
data_dir = './smoke_data'

# Check if data directory exists
if not os.path.exists(data_dir):
    print(f"Error: Data directory '{data_dir}' not found.")
    print("Please create a 'data' directory in the same location as this notebook and add the required data files.")
    required_files = [
        'CESM_09x125_PM25_2050_RCP85.nc',
        'CESM_09x125_PM25_2050_RCP85_NoFire.nc',
        'CESM_09x125_PM25_2000_Baseline.nc',
        'CESM_09x125_PM25_2000_BaseLine_NoFire.nc',
        'CESM_09x125_PM25_2100_RCP85.nc',
        'CESM_09x125_PM25_2100_RCP85_NoFire.nc',
        'death_rate.nc',
        'ssp2_2000_matching.nc',
        'ssp2_2050_matching.nc',
        'ssp2_2100_matching.nc'
    ]
    print(f"Required files: {', '.join(required_files)}")
    # Uncomment the line below if you want the notebook to stop execution when data is missing
    # sys.exit(1)

## Helper Functions

Define utility functions for data processing and visualization.

In [3]:
def fix_lon(ds):
    """Adjust longitude values to ensure they are within (-180, 180) range.
    
    Parameters:
        ds (xarray.Dataset): Dataset with longitude coordinates
        
    Returns:
        xarray.Dataset: Dataset with adjusted longitude coordinates
    """
    lon_name = 'lon'  # whatever name is in the data

    # Adjust lon values to make sure they are within (-180, 180)
    ds['_longitude_adjusted'] = xr.where(
        ds[lon_name] > 180,
        ds[lon_name] - 360,
        ds[lon_name])

    # reassign the new coords to as the main lon coords
    # and sort DataArray using new coordinate values
    ds = (
        ds
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(ds._longitude_adjusted)})
        .drop(lon_name))

    ds = ds.rename({'_longitude_adjusted': lon_name})
    return ds

In [4]:
def global_pcolormesh_country_borders(
    ds,
    title=None,
    cmap='Reds',
    cbar_label=None,
    cbar_ticks=None,
    cbar_ticklabels=None,
    cbar_orientation='vertical',
    vmin=None,
    vmax=None,
    bounds=None,
    colors=None,
    figsize=(10, 5),
):
    """
    Create a global pcolormesh plot with country borders.
    
    Parameters:
        ds (xarray.Dataset): The dataset containing the variable to plot.
        title (str): Title of the plot.
        cmap (str): Colormap to use.
        cbar_label (str): Label for the colorbar.
        cbar_ticks (list): Ticks for the colorbar.
        cbar_ticklabels (list): Tick labels for the colorbar.
        cbar_orientation (str): Orientation of the colorbar ('vertical' or 'horizontal').
        vmin (float): Minimum value for colormap scaling.
        vmax (float): Maximum value for colormap scaling.
        bounds (list): Boundaries for discrete colormap.
        colors (list): Colors for discrete colormap.
        figsize (tuple): Size of the figure.
    
    Returns:
        None
    """
    temp_ds = ds.copy()
    
    fig, ax = plt.subplots(figsize=figsize, subplot_kw={'projection': ccrs.PlateCarree()})

    if bounds is not None:
        norm = mcolors.BoundaryNorm(bounds, len(colors))
        cmap=mcolors.ListedColormap(colors)
        p = temp_ds.plot(
            ax=ax,
            transform=ccrs.PlateCarree(),
            cmap=cmap,
            norm=norm,
            add_colorbar=False
        )
    else:
        # Plotting the data
        p = temp_ds.plot(
            ax=ax,
            transform=ccrs.PlateCarree(),
            cmap=cmap,
            vmin=vmin,
            vmax=vmax,
            add_colorbar=False
        )
        
    # Adding country borders
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle='-', edgecolor='black', linewidth=0.5)

    # Adding title
    if title:
        plt.title(title)
    
    # Adding colorbar
    if cbar_label:
        cbar = plt.colorbar(p, ax=ax, orientation=cbar_orientation)
        cbar.set_label(cbar_label)
        
        if cbar_ticks is not None:
            cbar.set_ticks(cbar_ticks)
        
        if cbar_ticklabels is not None:
            cbar.ax.set_yticklabels(cbar_ticklabels)
    
    plt.show()

In [5]:
# Function to get the sum of values within each country and return as a pandas Series
def sum_within_country(data, countries):
    """Calculate the sum of values within each country's boundaries.
    
    Parameters:
        data (xarray.DataArray): Grid data to sum within country boundaries
        countries (list): List of country geometries
        
    Returns:
        pandas.Series: Series with country names as index and summed values
    """
    temp_data = data.copy()
    country_sums = []
    country_names = []
    
    # Get country info from shapefile
    shapefile = shpreader.natural_earth(resolution='110m', category='cultural', name='admin_0_countries')
    reader = shpreader.Reader(shapefile)
    country_info = list(reader.records())
    
    # Assuming data is an xarray.DataArray with 'lat' and 'lon' as coordinates
    lats = temp_data.coords['lat'].values
    lons = temp_data.coords['lon'].values
    
    for country, country_attr in zip(countries, country_info):
        clist = []
        country_name = country_attr.attributes['NAME']  # Get country name from the attributes
        country_polygon = shape(country)  # Convert country geometry to a shapely polygon
        
        # Create a meshgrid for latitude and longitude values
        lat_idx, lon_idx = np.meshgrid(lats, lons, indexing="ij")
        
        # Create an array of points
        points = np.array([Point(lon, lat) for lat, lon in zip(lat_idx.ravel(), lon_idx.ravel())])
        
        # Check which points are within the country's polygon
        inside_country = np.array([country_polygon.contains(point) for point in points])
        
        # Get the data values corresponding to the points inside the country
        inside_values = data.values.ravel()[inside_country]
        
        # Sum the values within the country
        country_sum = np.nansum(inside_values)
        
        # Append the country name and its corresponding sum to the lists
        country_sums.append(country_sum)
        country_names.append(country_name)
    
    # Return a pandas Series with the country name as the index and sum as the values
    return pd.Series(country_sums, index=country_names, name='Smoke Deaths')

In [6]:
def mortality(pm25, b=None, population=None):
    """Calculate mortality based on PM2.5 concentration, death rate, and population.
    
    Parameters:
        pm25 (xarray.DataArray): PM2.5 concentration
        b (xarray.DataArray): Death rate (per person)
        population (xarray.DataArray): Population count
        
    Returns:
        xarray.DataArray: Calculated mortality
    """
    temp_ds = b * (1-np.exp(-1.1 * (pm25))) * population
    temp_ds['lon'] = temp_ds['lon']
    return temp_ds

## Load Data

Load PM2.5 concentration data, death rates, and population projections.

In [7]:
# Load PM2.5 data for 2050
try:
    baseline_pm25_2050 = xr.open_dataset(os.path.join(data_dir, 'CESM_09x125_PM25_2050_RCP85.nc'))['pm25'].mean(axis=0)
    nofire_pm25_2050 = xr.open_dataset(os.path.join(data_dir, 'CESM_09x125_PM25_2050_RCP85_NoFire.nc'))['pm25'].mean(axis=0)
    baseline_pm25_2050 = fix_lon(baseline_pm25_2050)
    nofire_pm25_2050 = fix_lon(nofire_pm25_2050)
    print("Successfully loaded PM2.5 data for 2050")
except Exception as e:
    print(f"Error loading PM2.5 data for 2050: {e}")
    
# Load PM2.5 data for 2100
try:
    baseline_pm25_2100 = xr.open_dataset(os.path.join(data_dir, 'CESM_09x125_PM25_2100_RCP85.nc'))['pm25'].mean(axis=0)
    nofire_pm25_2100 = xr.open_dataset(os.path.join(data_dir, 'CESM_09x125_PM25_2100_RCP85_NoFire.nc'))['pm25'].mean(axis=0)
    baseline_pm25_2100 = fix_lon(baseline_pm25_2100)
    nofire_pm25_2100 = fix_lon(nofire_pm25_2100)
    print("Successfully loaded PM2.5 data for 2100")
except Exception as e:
    print(f"Error loading PM2.5 data for 2100: {e}")
    baseline_pm25_2100 = None
    nofire_pm25_2100 = None

Successfully loaded PM2.5 data for 2050
Successfully loaded PM2.5 data for 2100


/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))
/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))
/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))
/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))


In [8]:
# Load PM2.5 data for 2000
try:
    baseline_pm25_2000 = xr.open_dataset(os.path.join(data_dir, 'CESM_09x125_PM25_2000_Baseline.nc'))['pm25'].mean(axis=0)
    nofire_pm25_2000 = xr.open_dataset(os.path.join(data_dir, 'CESM_09x125_PM25_2000_Baseline_nofire.nc'))['pm25'].mean(axis=0)
    baseline_pm25_2000 = fix_lon(baseline_pm25_2000)
    nofire_pm25_2000 = fix_lon(nofire_pm25_2000)
    print("Successfully loaded PM2.5 data for 2000")
except Exception as e:
    print(f"Error loading PM2.5 data for 2000: {e}")

/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))


Successfully loaded PM2.5 data for 2000


/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))


In [9]:
# Load death rate and population data
try:
    death_rate = xr.open_dataset(os.path.join(data_dir, 'death_rate.nc'))['death_rate']/1000  # Convert to per person
    
    # Load population data for different years
    population_2000 = xr.open_dataset(os.path.join(data_dir, 'ssp2_2000_matching.nc'))['2000total']
    population_2050 = xr.open_dataset(os.path.join(data_dir, 'ssp2_2050_matching.nc'))['ssp2_2050']
    population_2100 = xr.open_dataset(os.path.join(data_dir, 'ssp2_2100_matching.nc'))['ssp2_2100']
    
    # Fix longitude coordinates
    death_rate = fix_lon(death_rate)
    population_2000 = fix_lon(population_2000)
    population_2050 = fix_lon(population_2050)
    population_2100 = fix_lon(population_2100)
    
    # Align coordinates with PM2.5 data
    death_rate['lat'] = baseline_pm25_2050['lat']
    death_rate['lon'] = baseline_pm25_2050['lon']
    
    for pop in [population_2000, population_2050, population_2100]:
        pop['lat'] = baseline_pm25_2050['lat']
        pop['lon'] = baseline_pm25_2050['lon']
    
    print("Successfully loaded death rate and population data for 2000, 2050, and 2100")
except Exception as e:
    print(f"Error loading death rate and population data: {e}")

/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))
/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))
/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))


Successfully loaded death rate and population data for 2000, 2050, and 2100


/var/folders/g9/z04gd5q11kv9sb76x89g7zym0000gp/T/ipykernel_3053/2969152750.py:24: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  .drop(lon_name))


## Calculate Mortality

Calculate mortality based on PM2.5 concentrations, death rates, and population.

In [10]:
# Calculate mortality for 2050 using actual 2050 population
baseline_mortality_2050 = mortality(baseline_pm25_2050, b=death_rate, population=population_2050)
nofire_mortality_2050 = mortality(nofire_pm25_2050, b=death_rate, population=population_2050)
difference_mortality_2050 = baseline_mortality_2050 - nofire_mortality_2050

print("Calculated mortality for 2050 scenarios using 2050 population")

Calculated mortality for 2050 scenarios using 2050 population


In [11]:
# Calculate mortality for 2000 using actual 2000 population
baseline_mortality_2000 = mortality(baseline_pm25_2000, b=death_rate, population=population_2000)
nofire_mortality_2000 = mortality(nofire_pm25_2000, b=death_rate, population=population_2000)
difference_mortality_2000 = baseline_mortality_2000 - nofire_mortality_2000

print("Calculated mortality for 2000 scenarios using 2000 population")

# Calculate mortality for 2100 if data is available
if baseline_pm25_2100 is not None and nofire_pm25_2100 is not None:
    baseline_mortality_2100 = mortality(baseline_pm25_2100, b=death_rate, population=population_2100)
    nofire_mortality_2100 = mortality(nofire_pm25_2100, b=death_rate, population=population_2100)
    difference_mortality_2100 = baseline_mortality_2100 - nofire_mortality_2100
    
    print("Calculated mortality for 2100 scenarios using 2100 population")
else:
    baseline_mortality_2100 = None
    nofire_mortality_2100 = None
    difference_mortality_2100 = None

Calculated mortality for 2000 scenarios using 2000 population
Calculated mortality for 2100 scenarios using 2100 population


## Aggregate by Country

Sum mortality values within each country's boundaries.

In [12]:
# Load countries shapefile
try:
    shapefile = shpreader.natural_earth(resolution='110m', category='cultural', name='admin_0_countries')
    reader = shpreader.Reader(shapefile)
    countries = list(reader.geometries())
    country_info = list(reader.records())
    print(f"Successfully loaded country boundaries from Natural Earth")
except Exception as e:
    print(f"Error loading country boundaries: {e}")

Successfully loaded country boundaries from Natural Earth


In [13]:
# Calculate country-level mortality for 2050
baseline_sums_2050 = sum_within_country(baseline_mortality_2050, countries)
nofire_sums_2050 = sum_within_country(nofire_mortality_2050, countries)
difference_sums_2050 = sum_within_country(difference_mortality_2050, countries)

# Save results to CSV
output_dir = os.path.join(data_dir, 'results')
os.makedirs(output_dir, exist_ok=True)

baseline_sums_2050.to_csv(os.path.join(output_dir, 'total_smoke_deaths.csv'))
nofire_sums_2050.to_csv(os.path.join(output_dir, 'nofire_smoke_deaths.csv'))
difference_sums_2050.to_csv(os.path.join(output_dir, 'fire_smoke_deaths.csv'))

print("Saved 2050 mortality results to CSV files")

Saved 2050 mortality results to CSV files


In [14]:
# Calculate country-level mortality for 2000
baseline_sums_2000 = sum_within_country(baseline_mortality_2000, countries)
nofire_sums_2000 = sum_within_country(nofire_mortality_2000, countries)
difference_sums_2000 = sum_within_country(difference_mortality_2000, countries)

# Save results to CSV
baseline_sums_2000.to_csv(os.path.join(output_dir, 'total_smoke_deaths_2000.csv'))
nofire_sums_2000.to_csv(os.path.join(output_dir, 'nofire_smoke_deaths_2000.csv'))
difference_sums_2000.to_csv(os.path.join(output_dir, 'fire_smoke_deaths_2000.csv'))

# Calculate change from 2000 to 2050
change_2000_2050 = difference_sums_2050 - difference_sums_2000
change_2000_2050.to_csv(os.path.join(output_dir, 'fire_smoke_deaths_change_2000_2050.csv'))

print("Saved 2000 mortality results and 2000-2050 changes to CSV files")

# Calculate country-level mortality for 2100 if data is available
if difference_mortality_2100 is not None:
    baseline_sums_2100 = sum_within_country(baseline_mortality_2100, countries)
    nofire_sums_2100 = sum_within_country(nofire_mortality_2100, countries)
    difference_sums_2100 = sum_within_country(difference_mortality_2100, countries)
    
    # Save 2100 results to CSV
    baseline_sums_2100.to_csv(os.path.join(output_dir, 'total_smoke_deaths_2100.csv'))
    nofire_sums_2100.to_csv(os.path.join(output_dir, 'nofire_smoke_deaths_2100.csv'))
    difference_sums_2100.to_csv(os.path.join(output_dir, 'fire_smoke_deaths_2100.csv'))
    
    # Calculate changes between time periods
    change_2050_2100 = difference_sums_2100 - difference_sums_2050
    change_2000_2100 = difference_sums_2100 - difference_sums_2000
    
    # Save changes to CSV
    change_2050_2100.to_csv(os.path.join(output_dir, 'fire_smoke_deaths_change_2050_2100.csv'))
    change_2000_2100.to_csv(os.path.join(output_dir, 'fire_smoke_deaths_change_2000_2100.csv'))
    
    print("Saved 2100 mortality results and changes to CSV files")

Saved 2000 mortality results and 2000-2050 changes to CSV files
Saved 2100 mortality results and changes to CSV files


## Summary Statistics

Display summary statistics for global mortality.

In [15]:
print(f"Total mortality in 2050 baseline scenario: {baseline_sums_2050.sum():,.0f}")
print(f"Total mortality in 2050 no-fire scenario: {nofire_sums_2050.sum():,.0f}")
print(f"Total mortality attributable to fire in 2050: {difference_sums_2050.sum():,.0f}")

print(f"\nTotal mortality attributable to fire in 2000: {difference_sums_2000.sum():,.0f}")
print(f"Change in fire-attributable mortality from 2000 to 2050: {(difference_sums_2050.sum() - difference_sums_2000.sum()):,.0f}")

if difference_mortality_2100 is not None:
    print(f"\nTotal mortality in 2100 baseline scenario: {baseline_sums_2100.sum():,.0f}")
    print(f"Total mortality in 2100 no-fire scenario: {nofire_sums_2100.sum():,.0f}")
    print(f"Total mortality attributable to fire in 2100: {difference_sums_2100.sum():,.0f}")
    
    print(f"\nChange in fire-attributable mortality from 2050 to 2100: {(difference_sums_2100.sum() - difference_sums_2050.sum()):,.0f}")
    print(f"Change in fire-attributable mortality from 2000 to 2100: {(difference_sums_2100.sum() - difference_sums_2000.sum()):,.0f}")

Total mortality in 2050 baseline scenario: 62,499,624
Total mortality in 2050 no-fire scenario: 62,191,602
Total mortality attributable to fire in 2050: 308,022

Total mortality attributable to fire in 2000: 137,055
Change in fire-attributable mortality from 2000 to 2050: 170,967

Total mortality in 2100 baseline scenario: 61,814,034
Total mortality in 2100 no-fire scenario: 61,018,132
Total mortality attributable to fire in 2100: 795,902

Change in fire-attributable mortality from 2050 to 2100: 487,880
Change in fire-attributable mortality from 2000 to 2100: 658,847


In [16]:
print(f"Total mortality attributable to fire in 2000: {difference_sums_2000.sum():,.0f}")
print(f"Change in fire-attributable mortality from 2000 to 2050: {(difference_sums.sum() - difference_sums_2000.sum()):,.0f}")

Total mortality attributable to fire in 2000: 137,055


NameError: name 'difference_sums' is not defined

## Visualizations

Create maps to visualize PM2.5 concentrations and mortality.

In [ ]:
# Visualize baseline PM2.5 concentrations 2050
global_pcolormesh_country_borders(baseline_pm25_2050, cbar_label='PM 2.5 Concentration',title='Total PM2.5 Concentration 2050')

In [ ]:
# Visualize fire contribution to PM2.5 2050
global_pcolormesh_country_borders(baseline_pm25_2050-nofire_pm25_2050, cbar_label='Fire PM 2.5 Concentration',title='Fire PM2.5 Concentration 2050')

In [ ]:
# Visualize fire contribution to PM2.5 in North America
global_pcolormesh_country_borders(
    baseline_pm25.loc[25:45,-140:-100]-nofire_pm25.loc[25:45,-140:-100],
    cbar_label='Fire PM 2.5 Concentration',
    title='Fire Contribution to PM2.5 in Western North America'
)

In [ ]:
# Visualize smoke deaths in 2050
diff_colors = ('white',"#fdcfbc", "#fc9676", "#f7593f", "#d21e20", "#67000d")
diff_bounds = [0,10,100,500,1000,5000]

global_pcolormesh_country_borders(
    difference_mortality_2050,
    title='Annual Smoke Deaths From Wildfire 2050',
    cbar_label='Annual Smoke Deaths',
    bounds=diff_bounds,
    colors=diff_colors
)

In [ ]:
# Visualize smoke deaths in 2000
global_pcolormesh_country_borders(
    difference_mortality_2000,
    title='Annual Smoke Deaths From Wildfire 2000',
    cbar_label='Annual Smoke Deaths',
    bounds=diff_bounds,
    colors=diff_colors
)

In [ ]:
# Visualize change in smoke deaths from 2000 to 2050
change_colors = ('blue', 'cyan', 'white', 'orange', 'red')
change_bounds = [-1000,-100,-10,10,100,1000]

global_pcolormesh_country_borders(
    difference_mortality_2050-difference_mortality_2000,
    title='Change in Annual Smoke Deaths From Wildfire 2000-2050',
    cbar_label='Change in Annual Smoke Deaths',
    bounds=change_bounds,
    colors=change_colors
)

## Country-Level Analysis

Examine the countries most affected by wildfire smoke mortality.

In [ ]:
# Top 10 countries with highest smoke deaths in 2050
top_countries_2050 = difference_sums_2050.sort_values(ascending=False).head(10)
print("Top 10 countries with highest smoke deaths in 2050:")
display(top_countries_2050)

In [ ]:
# Top 10 countries with highest increase in smoke deaths from 2000 to 2050
change_2000_2050 = difference_sums_2050 - difference_sums_2000
top_increase_2000_2050 = change_2000_2050.sort_values(ascending=False).head(10)
print("Top 10 countries with highest increase in smoke deaths from 2000 to 2050:")
display(top_increase_2000_2050)

# If 2100 data is available, show top countries for 2100 and changes
if difference_mortality_2100 is not None:
    # Top 10 countries with highest smoke deaths in 2100
    top_countries_2100 = difference_sums_2100.sort_values(ascending=False).head(10)
    print("\nTop 10 countries with highest smoke deaths in 2100:")
    display(top_countries_2100)
    
    # Top 10 countries with highest increase in smoke deaths from 2050 to 2100
    change_2050_2100 = difference_sums_2100 - difference_sums_2050
    top_increase_2050_2100 = change_2050_2100.sort_values(ascending=False).head(10)
    print("\nTop 10 countries with highest increase in smoke deaths from 2050 to 2100:")
    display(top_increase_2050_2100)

In [ ]:
# Add a new cell for visualizing time series data if 2100 data is available

if difference_mortality_2100 is not None:
    # Compare 2000, 2050, and 2100 for wildfire PM2.5 concentrations
    fig, axes = plt.subplots(1, 3, figsize=(18, 5), subplot_kw={'projection': ccrs.PlateCarree()})
    
    # Create a consistent colormap
    fire_cmap = plt.cm.Reds
    vmax = max(
        (baseline_pm25_2000-nofire_pm25_2000).max(),
        (baseline_pm25_2050-nofire_pm25_2050).max(),
        (baseline_pm25_2100-nofire_pm25_2100).max()
    )
    
    # 2000 fire contribution
    p1 = (baseline_pm25_2000-nofire_pm25_2000).plot(
        ax=axes[0],
        transform=ccrs.PlateCarree(),
        cmap=fire_cmap,
        vmin=0,
        vmax=vmax,
        add_colorbar=False
    )
    
    # 2050 fire contribution
    p2 = (baseline_pm25_2050-nofire_pm25_2050).plot(
        ax=axes[1],
        transform=ccrs.PlateCarree(),
        cmap=fire_cmap,
        vmin=0,
        vmax=vmax,
        add_colorbar=False
    )
    
    # 2100 fire contribution
    p3 = (baseline_pm25_2100-nofire_pm25_2100).plot(
        ax=axes[2],
        transform=ccrs.PlateCarree(),
        cmap=fire_cmap,
        vmin=0,
        vmax=vmax,
        add_colorbar=False
    )
    
    # Add features to all maps
    for ax, year in zip(axes, ["2000", "2050", "2100"]):
        ax.coastlines()
        ax.add_feature(cfeature.BORDERS, linestyle='-', edgecolor='black', linewidth=0.5)
        ax.set_title(f"Wildfire PM2.5 Contribution - {year}")
    
    # Add a common colorbar
    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    cbar = fig.colorbar(p3, cax=cbar_ax)
    cbar.set_label('Wildfire PM2.5 Contribution (μg/m³)', fontweight='bold')
    
    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.show()
    
    # Create a timeline plot showing global smoke deaths
    global_deaths = [
        difference_sums_2000.sum(),
        difference_sums_2050.sum(),
        difference_sums_2100.sum()
    ]
    
    plt.figure(figsize=(10, 6))
    plt.plot(['2000', '2050', '2100'], global_deaths, marker='o', linestyle='-', linewidth=2)
    
    # Add value labels
    for i, value in enumerate(global_deaths):
        plt.text(['2000', '2050', '2100'][i], value + 5000, f"{value:,.0f}", 
                 ha='center', va='bottom', fontweight='bold')
    
    plt.title('Global Wildfire-Related Mortality Over Time', fontsize=14, fontweight='bold')
    plt.ylabel('Annual Deaths', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Calculate and show percent changes
    pct_change_2000_2050 = (global_deaths[1] - global_deaths[0]) / global_deaths[0] * 100
    pct_change_2050_2100 = (global_deaths[2] - global_deaths[1]) / global_deaths[1] * 100
    
    plt.annotate(f"+{pct_change_2000_2050:.1f}%", 
                 xy=(0.5, (global_deaths[0] + global_deaths[1])/2),
                 xytext=(0.3, (global_deaths[0] + global_deaths[1])/2 + 20000),
                 arrowprops=dict(arrowstyle='->'),
                 fontsize=10, fontweight='bold')
    
    plt.annotate(f"+{pct_change_2050_2100:.1f}%", 
                 xy=(1.5, (global_deaths[1] + global_deaths[2])/2),
                 xytext=(1.3, (global_deaths[1] + global_deaths[2])/2 + 20000),
                 arrowprops=dict(arrowstyle='->'),
                 fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()